In [4]:
import pandas as pd

# Specify the file path
file_path = 'D:/UNT/Data Vis/DataViZ Project/UI/New Branch/dataviz/cleaned_dataset/InternationalSalesReport/Cleaned_InternationalSalesReport.csv'
# Read the CSV file into a dataframe
df = pd.read_csv(file_path)
df


,index,DATE,Months,CUSTOMER,Style,SKU,Size,PCS,RATE,GROSS AMT
0,0,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-L,L,1.00,616.56,617.00
1,1,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XL,XL,1.00,616.56,617.00
2,2,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5004,MEN5004-KR-XXL,XXL,1.00,616.56,617.00
3,3,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5009,MEN5009-KR-L,L,1.00,616.56,617.00
4,4,06-05-21,Jun-21,REVATHY LOGANATHAN,MEN5011,MEN5011-KR-L,L,1.00,616.56,617.00
...,...,...,...,...,...,...,...,...,...,...
34953,37427,AVIN,03-31-22,Mar-22,PJNE3423,PJNE3423-KR-4XL,1.00,537.50,538.00,4.00
34954,37428,AVIN,03-31-22,Mar-22,PJNE3404,PJNE3404-KR-4XL,1.00,500.00,500.00,5.00
34955,37429,AVIN,03-31-22,Mar-22,PJNE3423,PJNE3423-KR-4XL,1.00,537.50,538.00,4.00
34956,37430,AVIN,03-31-22,Mar-22,SET290,SET290-KR-DPT-M,1.00,812.50,812.00,7.00


In [5]:
import pandas as pd
import json

# Load your DataFrame
# df = pd.read_csv('your_data.csv')

# Assuming 'cleaned_df' is your DataFrame containing the data

# Filter out rows with unexpected values in the "Months" column
cleaned_df = df[df['Months'].str.match(r'^[A-Za-z]{3}-\d{2}$')]

# Convert the "Months" column to datetime format
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Months'], format='%b-%y')

# Convert 'GROSS AMT' column to numeric format
cleaned_df['GROSS AMT'] = pd.to_numeric(cleaned_df['GROSS AMT'])

# Create a mapping dictionary for month abbreviations to numeric values
month_mapping = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
                 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# Map the month abbreviations to numeric values
cleaned_df['NumericMonth'] = cleaned_df['Date'].dt.strftime('%b').map(month_mapping)

# Sort the DataFrame by numeric month
cleaned_df = cleaned_df.sort_values('NumericMonth')

# Group the dataframe by numeric month and year, and sum up the gross amount
monthly_gross_amount = cleaned_df.groupby(['NumericMonth', cleaned_df['Date'].dt.year])['GROSS AMT'].sum().sort_index()

# Convert the grouped data to a dictionary with string keys
monthly_gross_dict = {f"{month}-{year}": float(gross_amount) for (month, year), gross_amount in monthly_gross_amount.items()}

# Get unique month names and sort them in ascending order
months_in_order = sorted(set(cleaned_df['NumericMonth']))

# Serialize the dictionary to a single JSON file
with open('monthly_gross_amount.json', 'w') as json_file:
    json.dump({'input': months_in_order, 'output': list(monthly_gross_dict.values())}, json_file, indent=4)


C:\Users\lokes\AppData\Local\Temp\ipykernel_27412\25834560.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Date'] = pd.to_datetime(cleaned_df['Months'], format='%b-%y')
C:\Users\lokes\AppData\Local\Temp\ipykernel_27412\25834560.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['GROSS AMT'] = pd.to_numeric(cleaned_df['GROSS AMT'])
C:\Users\lokes\AppData\Local\Temp\ipykernel_27412\25834560.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [6]:
import pandas as pd
import json

# Convert 'GROSS AMT' column to numeric format
df['GROSS AMT'] = pd.to_numeric(df['GROSS AMT'], errors='coerce')

# Group the dataframe by unique styles and sum the gross amount
df_grouped = df.groupby('Style')['GROSS AMT'].sum().reset_index()

# Sort the grouped dataframe by the sum of the gross amount in descending order
df_grouped_sorted = df_grouped.sort_values(by='GROSS AMT', ascending=False)

# Select the top 15 rows
top_15 = df_grouped_sorted.head(15)

# Convert the selected dataframe to a list of dictionaries
result = top_15.to_dict(orient='records')

# Dump the result to a JSON file
with open('top_15_style_grossamount.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)


In [7]:
import json

# Group by 'size' and calculate the sum of 'gross_amount'
df_grouped = df.groupby('Size')['GROSS AMT'].sum().reset_index()

# Filter only the specified sizes
valid_sizes = ['4XL', '5XL', '6XL', 'FREE', 'L', 'M', 'PCS', 'S', 'XL', 'XS', 'XXL', 'XXXL']
df_filtered = df_grouped[df_grouped['Size'].isin(valid_sizes)]

# Convert the filtered DataFrame to a list of dictionaries
result = [{'data': [{ 'GROSS AMT': row['GROSS AMT'], 'Size': row['Size'] } for index, row in df_filtered.iterrows()]}]

# Dump the result to a JSON file
with open('sum_gross_amount_by_size.json', 'w') as json_file:
    for item in result:
        json_file.write(json.dumps(item, indent=4))


In [8]:
import pandas as pd

# Assuming df is your DataFrame and 'column_name' is the column for which you want to find unique values
unique_values_count = df['Size'].nunique()

print("Number of unique values in the column:", unique_values_count)


Number of unique values in the column: 30


In [9]:
import pandas as pd

# Assuming df is your DataFrame

# Convert 'GROSS AMT' column to numeric format
df['GROSS AMT'] = pd.to_numeric(df['GROSS AMT'], errors='coerce')

# Group the DataFrame by 'CUSTOMER' and 'Size' columns and sum the 'GROSS AMT' for each group
customer_size_gross_sum = df.groupby(['CUSTOMER', 'Size'])['GROSS AMT'].sum().reset_index()

# Create a dictionary to store the sums for each size for every unique customer
customer_size_dict = {}

# Iterate over each row in the grouped DataFrame
for index, row in customer_size_gross_sum.iterrows():
    customer = row['CUSTOMER']
    size = row['Size']
    gross_amt = row['GROSS AMT']
    
    # Check if the customer already exists in the dictionary
    if customer not in customer_size_dict:
        # If not, initialize an empty dictionary for the customer
        customer_size_dict[customer] = {}
    
    # Store the sum of gross amount for the size under the customer's dictionary
    customer_size_dict[customer][size] = gross_amt

# Print the resulting dictionary
print(customer_size_dict)


{'AANCHOL': {'L': 3818.0, 'XL': 1272.0, 'XXL': 3181.0, 'XXXL': 3181.0}, 'ABBI COLLECTION ': {'L': 9890.0, 'M': 2404.0, 'XL': 2550.0, 'XS': 1728.0, 'XXL': 13288.0, 'XXXL': 7254.0}, 'ABDUL RAHMAN(SINGAPORE)': {'L': 16086.0, 'XL': 14710.0, 'XXL': 16010.0, 'XXXL': 1300.0}, 'ADDVEN SOLUTIONS PRIVATE LIMITED': {'L': 18538.0, 'XS': 13690.0}, 'AKASH KAUSHAL': {'L': 574.0, 'M': 1091.0, 'S': 2212.0, 'XL': 649.0, 'XXXL': 1054.0}, 'ALAMELOO(MALA)': {'4XL': 1476.0, 'L': 15060.0, 'M': 5314.0, 'S': 3850.0, 'XL': 25993.0, 'XXL': 14248.0}, 'AMANI CONCEPT TRADING LLC (KAPDA)': {'L': 277586.0, 'M': 205823.0, 'XL': 290700.0, 'XXL': 82272.0, 'XXXL': 74070.0}, 'AMBIKA SYAM SUNDAR': {'L': 20698.0, 'M': 20479.0, 'S': 20442.0, 'XL': 19417.0, 'XXL': 19355.0, 'XXXL': 16688.0}, 'ANJU BHARATI': {'L': 42934.0, 'M': 47014.0, 'S': 51022.0, 'XL': 43552.0, 'XS': 37134.0, 'XXL': 20208.0}, 'ANU RANJANI ANPALAGAN': {'L': 8704.0, 'M': 8704.0, 'S': 8704.0, 'XL': 8704.0, 'XS': 4926.0, 'XXL': 8704.0, 'XXXL': 4926.0}, 'ARANGAN

In [10]:
# Iterate over each customer in the dictionary
unique_sizes = set()  # Initialize an empty set to collect unique sizes

for customer_sizes in customer_size_dict.values():
    # Iterate over each size in the customer's dictionary
    for size in customer_sizes.keys():
        unique_sizes.add(size)  # Add the size to the set of unique sizes

# Print the unique sizes
print("Unique Sizes:", unique_sizes)


Unique Sizes: {'9.00', '11.00', 'L', '10.00', '3.00', '4XL', '600.00', '12.00', 'XXXL', 'S', '4.00', 'XS', 'XXL', '6XL', '698.00', 'Free', '5XL', '7.00', 'M', 'PCS', '1000.00', '1.00', 'S TO XXL', '8.00', '6.00', 'FREE', 'XL', '5.00', '2.00', '15.00'}


In [11]:
valid_sizes = ['4XL', '5XL', '6XL', 'FREE', 'L', 'M', 'PCS', 'S', 'XL', 'XS', 'XXL', 'XXXL']

# Create a new dictionary to store the sums for each valid size for every unique customer
valid_customer_size_dict = {}

# Iterate over each customer in the original dictionary
for customer, customer_sizes in customer_size_dict.items():
    # Initialize an empty dictionary for the customer in the new dictionary
    valid_customer_size_dict[customer] = {}
    
    # Iterate over each size in the customer's dictionary
    for size in customer_sizes.keys():
        # Check if the size is valid
        if size in valid_sizes:
            # If valid, store the sum of gross amount for the size under the customer's dictionary
            valid_customer_size_dict[customer][size] = customer_sizes[size]

# Print the resulting dictionary containing only valid sizes
print(len(valid_customer_size_dict))


136


In [15]:
#{'4XL':8, '5XL':9, '6XL':10, 'FREE':0, 'L':5, 'M':4, 'PCS':11, 'S':3, 'XL':, 'XS':2, 'XXL':6, 'XXXL':7}

# Create a new dictionary to store the converted sizes for each customer
converted_customer_size_dict = {}

# Iterate over each customer in the original dictionary
for customer, customer_sizes in customer_size_dict.items():
    # Create a new dictionary for the converted sizes (keeping original sizes)
    converted_sizes = {}
    
    # Store the original sizes and counts in the new dictionary
    for size, count in customer_sizes.items():
        converted_sizes[size] = count
    
    # Store the converted sizes for the customer in the new dictionary
    converted_customer_size_dict[customer] = converted_sizes

# Print the resulting dictionary containing converted sizes
print(converted_customer_size_dict)


{'AANCHOL': {'L': 3818.0, 'XL': 1272.0, 'XXL': 3181.0, 'XXXL': 3181.0}, 'ABBI COLLECTION ': {'L': 9890.0, 'M': 2404.0, 'XL': 2550.0, 'XS': 1728.0, 'XXL': 13288.0, 'XXXL': 7254.0}, 'ABDUL RAHMAN(SINGAPORE)': {'L': 16086.0, 'XL': 14710.0, 'XXL': 16010.0, 'XXXL': 1300.0}, 'ADDVEN SOLUTIONS PRIVATE LIMITED': {'L': 18538.0, 'XS': 13690.0}, 'AKASH KAUSHAL': {'L': 574.0, 'M': 1091.0, 'S': 2212.0, 'XL': 649.0, 'XXXL': 1054.0}, 'ALAMELOO(MALA)': {'4XL': 1476.0, 'L': 15060.0, 'M': 5314.0, 'S': 3850.0, 'XL': 25993.0, 'XXL': 14248.0}, 'AMANI CONCEPT TRADING LLC (KAPDA)': {'L': 277586.0, 'M': 205823.0, 'XL': 290700.0, 'XXL': 82272.0, 'XXXL': 74070.0}, 'AMBIKA SYAM SUNDAR': {'L': 20698.0, 'M': 20479.0, 'S': 20442.0, 'XL': 19417.0, 'XXL': 19355.0, 'XXXL': 16688.0}, 'ANJU BHARATI': {'L': 42934.0, 'M': 47014.0, 'S': 51022.0, 'XL': 43552.0, 'XS': 37134.0, 'XXL': 20208.0}, 'ANU RANJANI ANPALAGAN': {'L': 8704.0, 'M': 8704.0, 'S': 8704.0, 'XL': 8704.0, 'XS': 4926.0, 'XXL': 8704.0, 'XXXL': 4926.0}, 'ARANGAN

In [16]:
import json
with open('Customers_Gross_Size.json', 'w') as file:
    json.dump(converted_customer_size_dict, file, indent=4)

In [17]:
import json

# Assuming `converted_customer_size_dict` contains your dictionary data

# Slice the dictionary to get the first 10 records
sliced_dict = dict(list(converted_customer_size_dict.items())[:10])

# Dump the sliced dictionary into the JSON file
with open('Customers_10_Gross_Size.json', 'w') as file:
    json.dump(sliced_dict, file, indent=4)
